In [5]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        print()

    def next(self):

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            if self.dataclose[0] > self.sma[0]:

                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()                
                


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    
    datapath = ('AAPL.csv')

    # Create a Data Feed
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        dtformat=('%Y-%m-%d'),
        tmformat=('%H.%M.%S'),
        fromdate=datetime.datetime(2000, 1, 1),
       
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=700)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()

Starting Portfolio Value: 100000.00
2019-10-01, BUY CREATE, 56.15
2019-10-02, BUY EXECUTED, Price: 55.76, Cost: 39035.50, Comm 0.00
2019-10-02, SELL CREATE, 54.74
2019-10-03, SELL EXECUTED, Price: 54.61, Cost: 39035.50, Comm 0.00
2019-10-03, OPERATION PROFIT, GROSS -810.25, NET -810.25

2019-10-03, BUY CREATE, 55.21
2019-10-04, BUY EXECUTED, Price: 56.41, Cost: 39487.00, Comm 0.00
2019-11-22, SELL CREATE, 65.44
2019-11-25, SELL EXECUTED, Price: 65.68, Cost: 39487.00, Comm 0.00
2019-11-25, OPERATION PROFIT, GROSS 6487.25, NET 6487.25

2019-11-25, BUY CREATE, 66.59
2019-11-26, BUY EXECUTED, Price: 66.74, Cost: 46714.50, Comm 0.00
2019-12-02, SELL CREATE, 66.04
2019-12-03, SELL EXECUTED, Price: 64.58, Cost: 46714.50, Comm 0.00
2019-12-03, OPERATION PROFIT, GROSS -1510.25, NET -1510.25

2019-12-05, BUY CREATE, 66.39
2019-12-06, BUY EXECUTED, Price: 66.87, Cost: 46809.00, Comm 0.00
2020-01-27, SELL CREATE, 77.24
2020-01-28, SELL EXECUTED, Price: 78.15, Cost: 46809.00, Comm 0.00
2020-01-28, 